# Data downloader


In [1]:
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q
!pip install datasets -q

     |████████████████████████████████| 7.6 MB 11.2 MB/s 
     |████████████████████████████████| 5.8 MB 2.7 MB/s 
     |████████████████████████████████| 182 kB 27.9 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 451 kB 15.5 MB/s 
     |████████████████████████████████| 132 kB 73.1 MB/s 
     |████████████████████████████████| 212 kB 61.5 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 


In [2]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
labels_names = dataset["train"].features["ner_tags"].feature.names

In [4]:
labels_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [5]:
dataset["train"][:2]

{'id': ['0', '1'],
 'tokens': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.'],
  ['Peter', 'Blackburn']],
 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7], [22, 22]],
 'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0], [11, 12]],
 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2]]}

# Data preparing

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding = "max_length", truncation = True, is_split_into_words = True)

In [8]:
tokenized_datasets_ = dataset.map(tokenize_function, batched = True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [9]:
len(tokenized_datasets_["train"][0]["input_ids"]) == len(tokenized_datasets_["train"][0]["ner_tags"])

False

Adjusting labels to fit input

In [10]:
def tokenize_adjust_labels(samples):
    tokenized_samples = tokenizer.batch_encode_plus(samples["tokens"], is_split_into_words = True, truncation = True)

    total_adjusted_labels = []

    for k in range(len(tokenized_samples["input_ids"])):
        prev_wid = -1

        word_ids_list = tokenized_samples.word_ids(batch_index = k)

        existing_label_ids = samples["ner_tags"][k]

        i = -1

        adjusted_label_ids = []

        for word_idx in word_ids_list:
            if word_idx is None:
                adjusted_label_ids.append(-100)
            elif word_idx != prev_wid:
                i += 1
                adjusted_label_ids.append(existing_label_ids[i])
                prev_wid = word_idx
            else:
                label_name = labels_names[existing_label_ids[i]]
                adjusted_label_ids.append(existing_label_ids[i])
        
        total_adjusted_labels.append(adjusted_label_ids)
            
    tokenized_samples["labels"] = total_adjusted_labels

    return tokenized_samples


In [11]:
out = tokenizer("Fine tune NER with BERT")
out

{'input_ids': [2, 1123, 6768, 13, 1031, 29, 11502, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
out.word_ids(0)

[None, 0, 1, 2, 2, 3, 4, None]

In [13]:
tokenized_dataset = dataset.map(tokenize_adjust_labels, batched = True, remove_columns = ["tokens", "ner_tags", "pos_tags", "chunk_tags"])

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [15]:
tokenized_dataset["train"][:2]

{'id': ['0', '1'],
 'input_ids': [[2, 2898, 12170, 18, 548, 645, 20, 16617, 388, 8624, 13, 9, 3],
  [2, 936, 14606, 3]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}

Padding token to a constant length

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

# Fine-tuning ALBERT for NER task

In [17]:
from transformers import AutoModelForTokenClassification

In [18]:
model = AutoModelForTokenClassification.from_pretrained("albert-base-v2", num_labels = len(labels_names))

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForTokenClassification: ['predictions.dense.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably

## Computing metrics 

In [19]:
import numpy as np
from datasets import load_metric

In [20]:
metric = load_metric("seqeval")

<ipython-input-20-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [21]:
def compute_metrics(p):
    predictions, labels = p

    predictions = np.argmax(predictions, axis = 2)

    true_predictions = [
        [labels_names[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [labels_names[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions = true_predictions, references = true_labels)

    return {
        "precision" : results["overall_precision"],
        "recall" : results["overall_recall"],
        "f1" : results["overall_f1"],
        "accuracy" : results["overall_accuracy"]
    }

## Using Trainer API to fine-tune

In [22]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

In [23]:
batch_size = 16
logging_steps = len(tokenized_dataset["train"]) // batch_size
epochs = 7
weight_decay = .01
eval_steps = 100
learning_rate = 2e-5
early_stopping_patience = 3

In [24]:
training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    disable_tqdm=False,
    learning_rate = learning_rate,
    logging_steps=logging_steps,
    evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = eval_steps,
    save_total_limit = 5,
    metric_for_best_model = "eval_loss",
    load_best_model_at_end = True
) 

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]
)

In [26]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.176483,0.798636,0.800132,0.799383,0.959318
200,No log,0.134922,0.844705,0.865285,0.854871,0.968391
300,No log,0.143632,0.854118,0.849410,0.851758,0.967468
400,No log,0.094277,0.883621,0.885569,0.884594,0.974881
500,No log,0.096906,0.862940,0.881490,0.872116,0.971828
600,No log,0.100743,0.883491,0.896153,0.889777,0.975648
700,No log,0.079697,0.882153,0.914342,0.897959,0.978530
800,No log,0.071836,0.886702,0.913681,0.899989,0.979368
900,0.135600,0.080888,0.882863,0.897365,0.890055,0.976925
1000,0.135600,0.070910,0.905011,0.915886,0.910416,0.979240


The following columns in the evaluation set don't have a corresponding argument in `AlbertForTokenClassification.forward` and have been ignored: id. If id are not expected by `AlbertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in results/checkpoint-1500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-1000 (score: 0.07091011852025986).


TrainOutput(global_step=1500, training_loss=0.10336282857259114, metrics={'train_runtime': 432.7734, 'train_samples_per_second': 227.11, 'train_steps_per_second': 14.201, 'total_flos': 54371670140832.0, 'train_loss': 0.10336282857259114, 'epoch': 1.71})

## Testing

In [27]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])

predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)

true_predictions = [
    [labels_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [labels_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)

The following columns in the test set don't have a corresponding argument in `AlbertForTokenClassification.forward` and have been ignored: id. If id are not expected by `AlbertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3453
  Batch size = 16


In [28]:
results

{'LOC': {'precision': 0.920019870839543,
  'recall': 0.8711194731890874,
  'f1': 0.8949021502778449,
  'number': 2126},
 'MISC': {'precision': 0.6573556797020484,
  'recall': 0.6827852998065764,
  'f1': 0.6698292220113853,
  'number': 1034},
 'ORG': {'precision': 0.8103571428571429,
  'recall': 0.8723567858515956,
  'f1': 0.8402147750416591,
  'number': 2601},
 'PER': {'precision': 0.9552017771195853,
  'recall': 0.9395484340859432,
  'f1': 0.9473104461171288,
  'number': 2746},
 'overall_precision': 0.8624825337680484,
 'overall_recall': 0.8706947219936523,
 'overall_f1': 0.8665691722725943,
 'overall_accuracy': 0.9686068717104752}

In [29]:
true_predictions

[['O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER', 'B-PER', 'I-PER', 'I-PER'],
 ['B-LOC',
  'B-LOC',
  'B-LOC',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'I-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'

In [30]:
true_labels

[['O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER', 'B-PER', 'I-PER', 'I-PER'],
 ['B-LOC',
  'B-LOC',
  'B-LOC',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'I-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'

In [31]:
from sklearn.metrics import classification_report

In [32]:
y_pred = []
y = []
for sent in true_predictions:
    for tag in sent:
        y_pred.append(tag)
for sent in true_labels:
    for tag in sent:
        y.append(tag)


In [33]:
print(classification_report(y_pred, y))

              precision    recall  f1-score   support

       B-LOC       0.87      0.93      0.90      2003
      B-MISC       0.73      0.76      0.75      1000
       B-ORG       0.90      0.85      0.88      2753
       B-PER       0.94      0.96      0.95      2696
       I-LOC       0.78      0.85      0.81       253
      I-MISC       0.75      0.35      0.47       540
       I-ORG       0.92      0.71      0.80      1372
       I-PER       0.97      0.98      0.98      2426
           O       0.98      0.99      0.99     51366

    accuracy                           0.97     64409
   macro avg       0.87      0.82      0.84     64409
weighted avg       0.97      0.97      0.97     64409



In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
print(confusion_matrix(y_pred, y))

[[ 1858    37    55    17     6     0     2     0    28]
 [   48   759    48    10     0    16     0     0   119]
 [  141   104  2346    76     0     0     7     1    78]
 [   14     4    53  2592     0     0     0    11    22]
 [    8     0     0     0   214     3    13     2    13]
 [    6    24     2     3     7   187    19     1   291]
 [   12     7    29     6    35    22   974    47   240]
 [    0     0     0    18     2     0    11  2384    11]
 [   39    99    68    24    10    23    30     0 51073]]
